### Experimenting With IIR Notch Filtering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastparquet, pyarrow
import mne
from mne.decoding import Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from mne_features.univariate import compute_pow_freq_bands
from utils import *

In [2]:
from scipy.signal import butter, filtfilt

In [3]:
mne.set_log_level('WARNING')

In [4]:
df = pd.read_csv('by_patient.csv')

other_df = activity_df(df, 'Other', 'expert_consensus')
seizure_df = activity_df(df, 'Seizure', 'expert_consensus')
gpd_df = activity_df(df, 'GPD', 'expert_consensus')
lpd_df = activity_df(df, 'LPD', 'expert_consensus')
grda_df = activity_df(df, 'GRDA', 'expert_consensus')
lrda_df = activity_df(df, 'LRDA', 'expert_consensus')

activity_df_list = [other_df, seizure_df, gpd_df, lpd_df, grda_df, lrda_df]

In [5]:
def iir_eeg(raw, n_channels, order, sfreq, l_freq, h_freq, b_type):
    filtered_eeg = pd.DataFrame()
    for i in range(n_channels):
        filtered_signal = iir_signal(raw.get_data()[i], order, sfreq, l_freq, h_freq, b_type)
        filtered_eeg = pd.concat([filtered_eeg, filtered_signal], axis = 1)
    filtered_eeg.columns = electrodes
    return filtered_eeg

def iir_signal(raw_signal, order, sfreq, l_freq, h_freq, b_type):
    nyquist = 0.5 * sfreq
    low = l_freq / nyquist
    high = h_freq / nyquist
    b, a = butter(order, [low, high], btype = b_type)
    filtered_signal = filtfilt(b, a, raw_signal)
    return pd.DataFrame(filtered_signal)